<a href="https://colab.research.google.com/github/LaviniBastos/Analise-Exploratoria-de-Dados/blob/main/Outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nessa atividade vamos entender um pouco mais como tratar pontos destoantes do conjunto, casos e exceções que podem prejudicar as análises, é chamado de **"Outliers"**.


In [1]:
import pandas as pd
import numpy as np


In [2]:
casa_alugar = pd.read_csv('./houses_to_rent.csv')

In [3]:
casa_alugar.head()

,city,area,rooms,bathroom,parking spaces,floor,hoa,rent,tax,insurance,total,page hits,days available,interactions,weeks available,type
0,São Paulo,70.0,2.0,1,1.0,7,2065,3300,211,42,5618.0,324,23,108,3,flat
1,São Paulo,320.0,4.0,4,2.0,20,1200,4960,1750,63,7973.0,720,78,240,11,flat
2,Porto Alegre,80.0,1.0,1,1.0,6,1000,2800,nr,41,3841.0,64,269,128,38,flat
3,Porto Alegre,51.0,1.0,1,NaN,2,270,1112,22,17,1421.0,46,73,92,10,flat
4,São Paulo,25.0,1.0,1,NaN,1,0,800,25,11,836.0,1548,78,516,11,flat


Vamos utilizar o describe para dar uma olhada nas estatisticas descritivas base desse **DataSet**

In [4]:
casa_alugar.describe()

,area,rooms,bathroom,parking spaces,hoa,rent,insurance,total,page hits,days available,interactions,weeks available
count,11687.000000,11686.000000,11765.000000,8800.000000,1.176500e+04,11765.000000,11765.000000,1.176500e+04,11765.000000,11765.000000,11765.000000,11765.000000
mean,145.652691,2.496406,2.280408,2.181705,1.162632e+03,4033.429154,56.525032,5.622370e+03,593.139992,73.644454,228.023969,10.307012
std,288.676311,1.165594,1.445071,1.519448,1.486720e+04,3564.190560,97.487374,1.579576e+04,559.727661,35.725794,174.183959,5.101312
min,1.000000,1.000000,1.000000,1.000000,0.000000e+00,200.000000,3.000000,2.635000e+02,0.000000,0.000000,0.000000,0.000000
25%,56.000000,2.000000,1.000000,1.000000,1.600000e+02,1600.000000,22.000000,2.176000e+03,144.000000,51.000000,100.000000,7.000000
50%,91.000000,2.000000,2.000000,2.000000,5.610000e+02,2790.000000,37.000000,3.762000e+03,432.000000,72.000000,180.000000,10.000000
75%,190.000000,3.000000,3.000000,3.000000,1.300000e+03,5100.000000,70.000000,6.990000e+03,900.000000,93.000000,312.000000,13.000000
max,24606.000000,10.000000,25.000000,12.000000,1.117000e+06,45000.000000,6000.000000,1.119892e+06,6444.000000,358.000000,2148.000000,51.000000


a partir daqui podemos ver que existe um numero *máximo de banheiros* que destoam da média da maioria.

In [5]:
casa_alugar['bathroom'].value_counts()

,count
bathroom,
1,4645
2,3105
3,1599
4,1438
5,614
6,252
7,92
8,11
9,4


De todo o conjunto de casas, existe somente **1 com 25 banheiros**, **1 com 20 banheiros**, e **3 casas com 10 banheiros**. Se quisermos calcular média de banheiros, esses números vão sem dúvidas impactar na qualidade da análise. Eles são os **"outliers"** nesse contexto

Existe alguns critérios para tratar os outliers, e o que será utilizado nesse exercício é o:

*IQR -> Distancia Inter quartil* (Inter Quantile range)

**Calculamos a distância entre o registro que está na posição 3/4
e o registro que está na posição 1/4**

- *Dados que são maiores que o 3/4 quartil + 1 x e meia essa distência = são outliers para mais*

- *Dados que são menores que o 1/4 quartil + 1 x e meia essa distância = são outliers para menores*


O **Pandas** tem uma função chamada **"quantile()"** que podemos apicar em uma certa coluna

In [7]:
q1 = casa_alugar['bathroom'].quantile(0.25) #Primeiro Quartil
q3 = casa_alugar['bathroom'].quantile(0.75) #Terceiro Quartil

IQR = q3 - q1

print(f"IQR: {IQR}")

IQR: 2.0


In [8]:
casas_outliers = casa_alugar[(casa_alugar['bathroom'] < q1 - (IQR * 1.5)) | (casa_alugar['bathroom'] > q3 + (IQR * 1.5))]

A partir desse critério o q3 está valendo 3.0:

In [10]:
q3

np.float64(3.0)

se verificarmos o valor do cálculo, temos:

In [11]:
q3 + (IQR * 1.5)

np.float64(6.0)

O valor é 6.
Então toda vez que aparecer quantidades de banheiro maior que 6, ele será considerado um outlier, um ponto fora da curva, e o filtro traz somente os pontos fora da curva, como no output abaixo:

In [9]:
casas_outliers

,city,area,rooms,bathroom,parking spaces,floor,hoa,rent,tax,insurance,total,page hits,days available,interactions,weeks available,type
17,Rio de Janeiro,88.0,2.0,20,1.0,9,1614,3500,221,16,5351.0,96,64,48,9,flat
1174,Campinas,NaN,5.0,7,8.0,-,0,14000,667,211,14878.0,148,80,148,11,house
1452,São Paulo,900.0,4.0,9,8.0,1,0,15000,5700,226,20926.0,1440,78,480,11,flat
1565,Belo Horizonte,548.0,4.0,7,6.0,17,5905,15000,2484,200,23589.0,162,87,162,12,flat
1671,Belo Horizonte,1000.0,5.0,7,8.0,-,0,14980,1904,246,17130.0,138,99,138,14,house
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11258,Campinas,560.0,7.0,7,8.0,-,0,9000,911,136,10047.0,52,86,52,12,house
11503,São Paulo,650.0,5.0,8,7.0,-,0,15000,2667,226,17893.0,396,85,132,12,house
11550,Belo Horizonte,330.0,5.0,7,5.0,10,1030,4800,973,64,6867.0,180,99,180,14,flat
11681,São Paulo,800.0,4.0,7,8.0,-,10000,8500,209,128,18837.0,1152,93,384,13,house


Agora podemos utilizar o mesmo filtro do outlier para ver os **"inliers"**. Aquele do primeiro quartil. inverteremos somente os operadores lógicos:

In [14]:
casas_inliers = casa_alugar[(casa_alugar['bathroom'] >= q1 - (IQR * 1.5)) & (casa_alugar['bathroom'] <= q3 + (IQR * 1.5))]

In [15]:
casas_inliers

,city,area,rooms,bathroom,parking spaces,floor,hoa,rent,tax,insurance,total,page hits,days available,interactions,weeks available,type
0,São Paulo,70.0,2.0,1,1.0,7,2065,3300,211,42,5618.0,324,23,108,3,flat
1,São Paulo,320.0,4.0,4,2.0,20,1200,4960,1750,63,7973.0,720,78,240,11,flat
2,Porto Alegre,80.0,1.0,1,1.0,6,1000,2800,nr,41,3841.0,64,269,128,38,flat
3,Porto Alegre,51.0,1.0,1,NaN,2,270,1112,22,17,1421.0,46,73,92,10,flat
4,São Paulo,25.0,1.0,1,NaN,1,0,800,25,11,836.0,1548,78,516,11,flat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11760,São Paulo,150.0,3.0,3,2.0,8,0,13500,0,172,13672.0,2124,114,708,16,flat
11761,Porto Alegre,63.0,2.0,1,1.0,5,402,1478,24,22,1926.0,58,86,116,12,flat
11762,São Paulo,285.0,4.0,4,4.0,17,3100,15000,973,191,19264.0,612,99,204,14,flat
11763,Brotas,100.0,1.0,2,1.0,-,0,800,116,39,955.0,148,127,148,18,house


In [16]:
casas_inliers.describe()

,area,rooms,bathroom,parking spaces,hoa,rent,insurance,total,page hits,days available,interactions,weeks available
count,11577.000000,11575.000000,11653.000000,8691.000000,1.165300e+04,11653.000000,11653.000000,1.165300e+04,11653.000000,11653.000000,11653.000000,11653.000000
mean,141.648095,2.472916,2.229984,2.138074,1.161767e+03,3975.474213,55.632369,5.550646e+03,592.354587,73.601905,227.836094,10.300953
std,286.410543,1.139837,1.340490,1.461525,1.493595e+04,3496.314911,97.227680,1.584018e+04,559.277132,35.647503,174.068174,5.090071
min,1.000000,1.000000,1.000000,1.000000,0.000000e+00,200.000000,3.000000,2.635000e+02,0.000000,0.000000,0.000000,0.000000
25%,56.000000,2.000000,1.000000,1.000000,1.800000e+02,1600.000000,22.000000,2.160000e+03,144.000000,51.000000,100.000000,7.000000
50%,90.000000,2.000000,2.000000,2.000000,5.700000e+02,2720.000000,36.000000,3.726000e+03,432.000000,72.000000,180.000000,10.000000
75%,183.000000,3.000000,3.000000,3.000000,1.300000e+03,5000.000000,69.000000,6.837000e+03,900.000000,93.000000,312.000000,13.000000
max,24606.000000,10.000000,6.000000,12.000000,1.117000e+06,30000.000000,6000.000000,1.119892e+06,6444.000000,358.000000,2148.000000,51.000000


Pelo output podemos ter uma média bem mais adequada! Comparada ao primeiro describe() que damos, e média estava em 2.28, e agora 2.22. Uma margem de quase 2% de erro que os outliers estavam gerando. O que não é um resultado tão extremo **neste caso**. Mas tratar os outliers nos permite ter mais **precisão** nas análises.